In [3]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from funkwpap import *
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys, matplotlib.pyplot as plt

In [5]:
x, y = sp.symbols('x y')
# Destroy Regulator and all the objectrs under it
if 'Regulator5' in locals():
  Regulator5.destroy_all()
  del Regulator5

# Create Sectors and Countries
Regulator5 = Regulator('EU', permit_price= 2, emission_cap= 100)
sector1 = Sector('cement', price_demand_function= 10-0.1*x, free_emission_multiplier= 0, regulator= Regulator5)
country1 = Country('DE', 1, regulator= Regulator5)


# Create Firms using objects
firm1 = Firm('firm1', sector1, country1, x*0, 2*x**2, 0, 0, 0, regulator= Regulator5)
firm2 = Firm('firm2', sector1, country1, x*0, x**2, 0, 0, 0, regulator= Regulator5)



print(sector1.firms)  # List of firms in sector1
print(country1.firms)  # List of firms in country1
print(Regulator5.firm_registry)  # Dictionary of all firms

for firm in Regulator5.firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name)

Cleaning up firm firm1
Cleaning up firm firm2
Cleaning up sector cement
Cleaning up country DE
EU regulator destroyed
[Firm(id=5, name='firm1', sector_id=3, country_id=3, actual_output=0, emission=0, profit=0), Firm(id=6, name='firm2', sector_id=3, country_id=3, actual_output=0, emission=0, profit=0)]
[Firm(id=5, name='firm1', sector_id=3, country_id=3, actual_output=0, emission=0, profit=0), Firm(id=6, name='firm2', sector_id=3, country_id=3, actual_output=0, emission=0, profit=0)]
{5: Firm(id=5, name='firm1', sector_id=3, country_id=3, actual_output=0, emission=0, profit=0), 6: Firm(id=6, name='firm2', sector_id=3, country_id=3, actual_output=0, emission=0, profit=0)}
firm1 cement DE
firm2 cement DE


In [23]:
pp = sp.symbols('pp') # permit price
q1, q2 = sp.symbols('q1 q2') # quantity of firm1 and firm2
x1, x2 = sp.symbols('x1 x2') # emission of firm1 and firm2
cap = 50

profit1 = (10 - 0.1 * (q1 + q2)) * q1 - 2*(q1 - x1)**2 - pp*x1
profit2 = (10 - 0.1 * (q1 + q2)) * q2 - (q2 - x2)**2 - pp*x2

# First order conditions
foc1 = sp.diff(profit1, q1)
foc2 = sp.diff(profit2, q2)
foc3 = sp.diff(profit1, x1)
foc4 = sp.diff(profit2, x2)

# Solve for q1, q2, x1, x2
solutions = sp.solve([foc1, foc2, foc3, foc4, x1 + x2 - 50], (q1, q2, x1, x2, pp))

print(solutions)

# Test the equilibrium using the calculate_output method of the firm.
Regulator5.permit_price = solutions.get(pp)
firm1.actual_output = solutions.get(q1)
firm2.actual_output = solutions.get(q2)
firm1.emission = solutions.get(x1)
firm2.emission = solutions.get(x2)

firm1.calculate_output(verbose = True)



{pp: 2.24719101123596, q1: 25.8426966292135, q2: 25.8426966292135, x1: 25.2808988764045, x2: 24.7191011235955}
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-9900 CPU @ 3.10GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3 rows, 6 columns and 7 nonzeros
Model fingerprint: 0x538ad382
Model has 2 general constraints
Variable types: 6 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective -0.0000000
Presolve added 31 rows and 266 columns
Presolve time: 0.00s
Presolved: 34 rows, 272 columns, 3857 nonzeros
Variable types: 258 continuous, 14 integer (2 binary)

Root relaxation: objective 2.263775e+04, 58 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    

(484.44666502756394, 482.2092172840317, 22637.74925257104)

In [30]:
pp = sp.symbols('pp') # permit price
q1, q2 = sp.symbols('q1 q2') # quantity of firm1 and firm2
x1, x2 = sp.symbols('x1 x2') # emission of firm1 and firm2
cap = 50

profit1 = (10 - 0.1 * (q1 + q2)) * q1 - 2*(q1 - x1)**3 - pp*x1
profit2 = (10 - 0.1 * (q1 + q2)) * q2 - (q2 - x2)**2 - pp*x2

# First order conditions
foc1 = sp.diff(profit1, q1)
foc2 = sp.diff(profit2, q2)
foc3 = sp.diff(profit1, x1)
foc4 = sp.diff(profit2, x2)

# Solve for q1, q2, x1, x2
solutions = sp.solve([foc1, foc2, foc3, foc4, x1 + x2 - 50], (q1, q2, x1, x2, pp))

print(solutions)


[(25.8656062689302, 25.8656062689302, 25.2545527907303, 24.7454472092697, 2.24031811932095), (25.2863677710915, 25.2863677710915, 25.9206770632448, 24.0793229367552, 2.41408966867256)]


In [46]:
pp = sp.symbols('pp') # permit price
q1, q2 = sp.symbols('q1 q2') # quantity of firm1 and firm2
x1, x2 = sp.symbols('x1 x2') # emission of firm1 and firm2
cap = 50

profit1 = (10 - 0.1 * (q1 + q2)) * q1 - 2*(q1 - x1)**3 - pp*x1
profit2 = (10 - 0.1 * (q1 + q2)) * q2 - (q2 - x2)**2 - pp*x2

# First order conditions
foc1 = sp.diff(profit1, q1)
foc2 = sp.diff(profit2, q2)
foc3 = sp.diff(profit1, x1)
foc4 = sp.diff(profit2, x2)

# Solve for q1, q2, x1, x2, pp numerically
initial_guess = (1, 1, 1, 1, 1)  # Adjust the initial guess as needed
solutions = sp.nsolve([foc1, foc2, foc3, foc4, x1 + x2 - 50], (q1, q2, x1, x2, pp), initial_guess)

print(solutions)

Matrix([[25.8656062689302], [25.8656062689302], [25.2545527907303], [24.7454472092697], [2.24031811932095]])


In [65]:
import numpy as np
from scipy.optimize import fsolve, root
import sympy as sp

# Define the symbols
q1, q2, x1, x2, pp = sp.symbols('q1 q2 x1 x2 pp')

# Define the equations symbolically
cap = 50
profit1 = (10 - 0.1 * (q1 + q2)) * q1 - 100*(q1 - x1)**3 - pp*x1
profit2 = (10 - 0.1 * (q1 + q2)) * q2 - 100*(q2 - x2)**2 - pp*x2

foc1 = sp.diff(profit1, q1)
foc2 = sp.diff(profit2, q2)
foc3 = sp.diff(profit1, x1)
foc4 = sp.diff(profit2, x2)

# Define the constraint
constraint = x1 + x2 - cap

# Convert the symbolic expressions to numerical functions
foc1_func = sp.lambdify((q1, q2, x1, x2, pp), foc1, 'numpy')
foc2_func = sp.lambdify((q1, q2, x1, x2, pp), foc2, 'numpy')
foc3_func = sp.lambdify((q1, q2, x1, x2, pp), foc3, 'numpy')
foc4_func = sp.lambdify((q1, q2, x1, x2, pp), foc4, 'numpy')
constraint_func = sp.lambdify((q1, q2, x1, x2, pp), constraint, 'numpy')

# Define the equations for fsolve
def equations(vars):
    q1_val, q2_val, x1_val, x2_val, pp_val = vars
    return [
        foc1_func(q1_val, q2_val, x1_val, x2_val, pp_val),
        foc2_func(q1_val, q2_val, x1_val, x2_val, pp_val),
        foc3_func(q1_val, q2_val, x1_val, x2_val, pp_val),
        foc4_func(q1_val, q2_val, x1_val, x2_val, pp_val),
        constraint_func(q1_val, q2_val, x1_val, x2_val, pp_val)
    ]

# Initial guess
initial_guess = [1, 1, 1, 1, 1]

print(foc1)

# Solve the equations
solutions = fsolve(equations, initial_guess)

print(solutions)

-0.2*q1 - 0.1*q2 - 300*(q1 - x1)**2 + 10
[24.96052809 24.96052809 25.05203112 24.94796888  2.51184157]
